### Downloading Datamart Snapshot

In [9]:
%%bash

curl -LO https://auctus.vida-nyu.org/snapshot/index.tar.gz
if [ -e index.snapshot ]; then
    rm -rf index.snapshot;
fi
mkdir index.snapshot
tar xfC index.tar.gz index.snapshot

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70.1M  100 70.1M    0     0  2249k      0  0:00:31  0:00:31 --:--:-- 1923k


### Choosing Datasets from Snapshot

In [11]:
import json
import os
import random
import shutil

input_path = 'index.snapshot'
output_path = 'random.sample'

if os.path.exists(output_path):
    shutil.rmtree(output_path)
os.mkdir(output_path)

input_path_datasets = os.listdir(input_path)
datasets = [[d] for d in input_path_datasets if d.startswith('datamart.socrata.data-cityofnewyork-us')]
for i in range(len(datasets)):
    for d in input_path_datasets:
        if d.startswith('lazo.%s'%datasets[i][0]):
            datasets[i].append(d)
random.shuffle(datasets)

datasets_to_upload = list()
for dataset in datasets:
    dataset_path = os.path.join(input_path, dataset[0])
    metadata = json.load(open(dataset_path))

    # ignoring datasets bigger than 50 MB
    if metadata["size"] >= 50000000:
        continue

    datasets_to_upload.append(dataset)

print("Size:", len(datasets_to_upload))

for dataset in datasets_to_upload:
    for d in dataset:
        shutil.copyfile(
            os.path.join(input_path, d),
            os.path.join(output_path, d)
        )

Size: 1874


In [12]:
%%bash

rm -rf index.snapshot

### Importing to Datamart (Local Deployment)

In [13]:
%%bash

DATAMART_DIRECTORY=/Users/fchirigati/projects/d3m/datamart
SNAPSHOT=$(pwd)/random.sample
AMQP_USER=datamart
AMQP_PASSWORD=datamart

cd $DATAMART_DIRECTORY
PROJ="$(basename "$(pwd)")"
docker run --rm --network ${PROJ}_default -v $PWD/scripts:/scripts -v $SNAPSHOT:/snapshot -e ELASTICSEARCH_HOSTS=elasticsearch:9200 -e AMQP_HOST=rabbitmq -e AMQP_USER=$AMQP_USER -e AMQP_PASSWORD=$AMQP_PASSWORD -w /tmp ${PROJ}_coordinator sh -c 'python /scripts/import_all.py /snapshot'


Importing Elasticsearch data............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

